In [ ]:
!pip install -q imageio
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
import pandas as pd
import tensorflow_docs.vis.embed as embed
from PIL import Image
from tensorflow.keras import layers
import time
from IPython import display

## Load data

In [ ]:
data_dir = '../input/celeba-dataset/'

In [ ]:
df_partition = pd.read_csv(f'{data_dir}list_eval_partition.csv')
img_dir = f'{data_dir}img_align_celeba/img_align_celeba/'
train_paths = [img_dir + i for i in df_partition[df_partition['partition'] == 0]['image_id']]
val_paths = [img_dir + i for i in df_partition[df_partition['partition'] == 1]['image_id']]
test_paths = [img_dir + i for i in df_partition[df_partition['partition'] == 2]['image_id']]
print(f'Train: {len(train_paths)}')
print(f'Val: {len(val_paths)}')
print(f'Test: {len(test_paths)}')

In [ ]:
def load_image(path, max_dim=None, gray=False):
    img = Image.open(path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    if gray:
        img = img.convert('L')
    return np.array(img)

In [ ]:
def normalize(img):
    img = (img - 127.5) / 127.5
    return img


def denormalize(img):
    img = img * 127.5 + 127.5
    return img


def show(img, gray=False):
    plt.axis('off')
    if gray:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img)

In [ ]:
# all valid dimentions
for i in range(max(load_image(train_paths[100000], gray=True).shape)):
    im = load_image(train_paths[100000], max_dim=i, gray=True)
    shape = im.shape
    if shape[0] % 4 == 0 and shape[1] % 4 == 0:
        print(shape)

In [ ]:
max_dim = 88
gray = False

In [ ]:
im = load_image(train_paths[100000], max_dim=max_dim, gray=gray)
shape = im.shape
print(f'Shape: {shape}')
# Both dimentions have to be divisible by 4
assert shape[0] % 4 == 0
assert shape[1] % 4 == 0
show(im, gray)

In [ ]:
train_images = np.array([load_image(path, max_dim=max_dim, gray=gray)
                         for path in test_paths])
if len(train_images.shape) == 3:
    train_images = train_images.reshape(train_images.shape + (1,))
input_shape = train_images.shape[1:]
train_images.shape

In [ ]:
train_images = normalize(train_images)

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# Delete train_images to save memory
del train_images

## The Generator

In [ ]:
noise_dim = 100

In [ ]:
def make_generator_model(img_shape):
    model = tf.keras.Sequential()
    model.add(layers.Dense((img_shape[0] // 4) * (img_shape[1] // 4) * 256, 
                           use_bias=False, input_shape=(noise_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((img_shape[0] // 4, img_shape[1] // 4, 256)))
    assert model.output_shape == (None, img_shape[0] // 4, img_shape[1] // 4, 256)

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, img_shape[0] // 4, img_shape[1] // 4, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, img_shape[0] // 2, img_shape[1] // 2, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(img_shape[2], (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, img_shape[0], img_shape[1], img_shape[2])

    return model

In [ ]:
generator = make_generator_model(input_shape)
generator.summary()

In [ ]:
noise = tf.random.normal([1, noise_dim])
generated_image = generator(noise, training=False)

show(generated_image[0, :, :, :]/2+0.5, gray)

## The Discriminator

In [ ]:
def make_discriminator_model(img_shape):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=list(img_shape)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model(input_shape)
discriminator.summary()

In [ ]:
decision = discriminator(generated_image)
print(decision)

## Define the loss and optimizers

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop

In [ ]:
EPOCHS = 500
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        # Produce images for the GIF as you go
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print(f'Time for epoch {epoch + 1} is {time.time() - start} sec')

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(8, 8))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        pred = predictions[i, :, :, :] / 2 + 0.5
#         pred = predictions[i, :, :, :]
#         pred = denormalize(predictions[i, :, :, :])
        if gray:
            plt.imshow(pred, cmap='gray')
        else:
            plt.imshow(pred)
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

## Train model

In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create GIF

In [ ]:
def display_image(epoch_no):
    return Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob('image*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

In [ ]:
embed.embed_file(anim_file)